In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from db_utils import sp2df

In [3]:
from asteroid_element import get_ast_elts, make_sim_asteroids
from orbital_element import anomaly_E2f, anomaly_f2E, anomaly_E2M, danby_guess, danby_iteration, elt2pos
from orbital_element_test import make_test_elements, get_test_elements, angle_distance, report_test, test_E2f, test_E2M, test_M2E, test_all
from asteroid_data import ast_add_sun_vectors

In [4]:
from planets_interp import get_earth_pos, get_earth_vectors, get_earth_elt, get_sun_vectors

In [5]:
# Get test orbital elements
elts = get_test_elements()

# Add vectors for the sun
ast_add_sun_vectors(elts)

In [6]:
elts

,TimeID,AsteroidID,mjd,qx,qy,qz,vx,vy,vz,a,...,Omega,omega,f,M,sun_qx,sun_qy,sun_qz,sun_vx,sun_vy,sun_vz
0,84960000,1,59000.0,2.200915,-1.931843,-0.467561,0.006340,0.007130,-0.000945,2.767657,...,1.401272,1.286860,-3.401546,2.839411,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
1,84960000,2,59000.0,0.662691,-2.706223,1.817727,0.008356,0.000283,-0.000904,2.773841,...,3.019851,-0.869132,2.742191,2.530303,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
2,84960000,3,59000.0,-2.901474,-1.192233,0.390143,0.001943,-0.008331,0.001812,2.668285,...,2.964468,-1.953614,2.535135,2.189260,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
3,84960000,4,59000.0,-0.240387,2.551044,-0.047390,-0.010162,-0.001270,0.001274,2.362014,...,1.811810,2.633265,-2.783176,-2.716995,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
4,84960000,5,59000.0,-2.063769,0.498221,0.084128,-0.003827,-0.012194,0.001120,2.574037,...,2.470881,-0.023590,0.461775,0.311477,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,84960000,9995,59000.0,1.929299,1.989779,0.047063,-0.006940,0.006451,0.000343,2.390249,...,0.363924,-2.826248,3.260392,-2.979609,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
9989,84960000,9996,59000.0,-0.850272,-2.870477,0.281759,0.009476,-0.000501,0.000683,2.796605,...,-2.629937,2.898821,-2.120904,-1.682728,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
9990,84960000,9997,59000.0,-2.235940,-1.662699,-0.104958,0.005379,-0.008184,-0.000224,2.545674,...,-0.427796,1.579063,-3.649961,2.514484,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
9991,84960000,9998,59000.0,1.949042,-0.287275,0.015234,0.001389,0.012640,0.000860,2.160961,...,-0.262023,0.578132,-0.465334,-0.386152,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07


In [7]:
# test_all()

In [8]:
# Unpack orbital elements
a = elts.a.values
e = elts.e.values
inc = elts.inc.values
Omega = elts.Omega.values
omega = elts.omega.values
f = elts.f.values
M = elts.M.values

In [9]:
# Position of the sun
cols_q_sun = ['sun_qx', 'sun_qy', 'sun_qz']
q_sun = elts[cols_q_sun].values

In [10]:
# Calculate the distance from the center, r; SSD equation 2.20
r = a * (1.0 - np.square(e)) / (1.0 + e * np.cos(f))

# Calculate intermediate results used for angular rotations
# The angle in the elliptic plane, measured from the reference direction
theta = omega + f 
# Trigonometric functions of the angles
cos_inc = np.cos(inc)
sin_inc = np.sin(inc)
cos_Omega = np.cos(Omega)
sin_Omega = np.sin(Omega)
cos_theta = np.cos(theta)
sin_theta = np.sin(theta)

In [11]:
# The cartesian position coordinates; see SSD equation 2.122
qx = r * (cos_Omega*cos_theta - sin_Omega*sin_theta*cos_inc)
qy = r * (sin_Omega*cos_theta + cos_Omega*sin_theta*cos_inc)
qz = r * (sin_theta*sin_inc)

In [12]:
# Reconstructed position
q_hel = np.stack([qx, qy, qz], axis=1)
q2 = q_hel + q_sun

In [13]:
# The position according to the integration
cols_q = ['qx', 'qy', 'qz']
q: np.array = elts[cols_q].values

In [14]:
# Position error
dq = q2 - q
err: np.array = np.sqrt(np.sum(np.square(dq), axis=1))
report_test(err, 'elt2pos', 1.0E-9)


Test: elt2pos
RMS error: 5.283e-14
Max error: 4.464e-12
*** PASS ***


In [15]:
# The gravitational constant in unit system (years, AU, Msun)
# numerical value close to 4 pi^2; see rebound documentation for exact value        
G_ = 39.476926421373
mu = G_ * 1.0

# The speed
one_minus_e2 = 1.0 - np.square(e)
v0 = np.sqrt(mu / a / one_minus_e2)

In [17]:
# Velocity
# vx = v0*((e+cf)*(-ci*co*sO - cO*so) - sf*(co*cO - ci*so*sO))
# vy = v0*((e+cf)*(ci*co*cO - sO*so)  - sf*(co*sO + ci*so*cO))
# vz = v0*((e+cf)*co*si - sf*si*so)
# vx = v0 * ()

In [18]:
epcos_f = np.cos(f)


In [19]:
from orbital_element import tau, anomaly_M2E_danby, interp_M2E, anomaly_M2E

In [20]:
E = anomaly_M2E_danby(M, e)
E

array([2.86089618, 2.64073125, 2.36866221, ..., 2.57515446, 5.75798806,
       5.53882808])

In [21]:
E2 = interp_M2E.ev(M, e) % tau

In [22]:
E2

array([2.86089618e+00, 2.64073125e+00, 2.36866221e+00, ...,
       2.57515446e+00, 5.22312761e-20, 2.41446786e-20])

In [23]:
err = angle_distance(E, E2)

In [26]:
idx = np.argmax(err)

5394

In [27]:
elts.iloc[idx]

2.0337550613296607

In [29]:
E2[idx]

6.283185307179586